In [1]:
import os
from ocr_module import extract_text

## OCR - Optical Character Recognision
#### Using: Google vision ocr 
##### google_vision requires credentials. Alternative: "surya"
##### Open source options are very poor




In [2]:
image_path = "temp_images/example1_s3.jpeg"
with open(image_path, "rb") as f:
            img_bytes = f.read()
extracted_text = extract_text(img_bytes, ocr_engine="google_vision")
print("\n\n\n")
print("Extracted Text:")
print("-"*20)
print(extracted_text)
print("-"*20)

Initializing Google Vision Client...
Google Vision Client Initialized.
Calling Google Cloud Vision OCR (Attempt 1/3)...


Google Vision OCR generated 95 lines (paragraphs).
Median line height: 37.00, Y-tolerance: 25.90




Extracted Text:
--------------------
WHOLESALE
Main Street Wholesale Meats INVOICE
Farmingdale Meat Market
FARMINGDALE 210 Main Street
MARKET Farmingdale , NY 11735 Invoice No. Invoice Date Page
MEAT ( 516 ) 249-8200 915378 08/07/24 PO Number 1
1
SOLD THE CHEQUIT
9 WEST 57TH STREET SHIP THE CHEQUIT
SUITE 4500 23 GRAND AVENUE
NEW YORK , NY 10019 SHELTER ISLAND HGTS , NY 11965
PHONE : ( 631 ) 749-0018 70 T
FAX :
PHONE :
Route Stop
NF Sales ID Customer ID Order No. 1415912 Order Date 08/06/24 Ship Via
Terms
VB CHEQUIT A14
Item ID
Qty Shipped Description Weight Price Per Total
BFGR
60ZFLTMGNCHN 10.00 LB USDA CHOICE GROUND BEEF 10.00 4.89 LB 48.90
PK - LN - BNLS 32.00 EA 6OZ FILET MIGNON STEAK USDA CHOICE C / C 13.05 33.99 LB 443.57
KIEL 2.00 EA LOIN OF PORK C / C BONELESS CRY - O 23.70 2.89 LB 68.49
10.00 LB SMOKED KIELBASA 11.70 8.99 LB 105.18
SAB FRNKS
6.00 LB SABRETT 8-1 SKINLESS FRANKS 

## Classifier
##### Tool/ function to give more context to the Agent.
##### Cheaper/Smaller Model

In [3]:
from classifier import classify_and_suggest_fields

In [4]:
classifier_result = classify_and_suggest_fields(extracted_text, img_bytes)

Sending classification prompt with model: classifier
Using model: Qwen2.5-7B-Instruct


Received classification response from LLM
Parsing classification response
Parsed classification as JSON: type='invoice', fields=['buyer_address', 'buyer_name', 'buyer_vat_number', 'currency', 'invoice_amount', 'invoice_date', 'invoice_number', 'payment_due_date', 'po_number', 'seller_address', 'seller_email', 'seller_fax_number', 'seller_name', 'seller_phone', 'seller_vat_number', 'seller_website', 'shipping_date', 'shipto_address', 'shipto_name', 'subtotal', 'total_due_amount', 'total_tax', 'terms_and_conditions', 'credit_service_charge', 'surcharge', 'returns_policy', 'claim_policy', 'misc_fees']


In [27]:
print(classifier_result['doc_type'])

invoice


## Iteration 1
####1. Extractor LLM: Qwen2.5 7B
#####Decoder only LLM

In [4]:
from llm_module import llm_call
from utils.extractor_utils import create_extraction_prompt_messages_text, parse_extraction

In [6]:
extracted_fields = ["buyer_address", "buyer_name", "buyer_vat_number", "currency", "invoice_amount", "invoice_date", "invoice_number", "payment_due_date", "po_number", "seller_address", "seller_email", "seller_fax_number", "seller_name", "seller_phone", "seller_vat_number", "seller_website", "shipping_date", "shipto_address", "shipto_name", "subtotal", "total_due_amount", "total_tax"]#TODO: change name

In [30]:
messages = create_extraction_prompt_messages_text(extracted_fields, extracted_text)
response = llm_call(messages, model_identifier="qwen_25")
extraction_result = parse_extraction(response, extracted_fields)
print("Extraction Result:")
print("-"*20)
print(extraction_result)
print("-"*20)

Using model: Qwen2.5-7B-Instruct


Extraction Result:
--------------------
{'buyer_address': '9 WEST 57TH STREET SUITE 4500 23 GRAND AVENUE NEW YORK , NY 10019 SHELTER ISLAND HGTS , NY 11965', 'buyer_name': 'THE CHEQUIT', 'buyer_vat_number': '', 'currency': 'USD', 'invoice_amount': '749.37', 'invoice_date': '08/07/24', 'invoice_number': '915378', 'payment_due_date': '', 'po_number': '1', 'seller_address': 'FARMINGDALE 210 Main Street MARKET Farmingdale , NY 11735', 'seller_email': '', 'seller_fax_number': '', 'seller_name': 'Main Street Wholesale Meats', 'seller_phone': '( 631 ) 749-0018', 'seller_vat_number': '', 'seller_website': '', 'shipping_date': '', 'shipto_address': '9 WEST 57TH STREET SUITE 4500 23 GRAND AVENUE NEW YORK , NY 10019 SHELTER ISLAND HGTS , NY 11965', 'shipto_name': 'THE CHEQUIT', 'subtotal': '742.37', 'total_due_amount': '749.37', 'total_tax': '7.00'}
--------------------


In [31]:
#Printing Relevant Results

print(f"Invoice Number: \n{extraction_result['invoice_number']}\n")

print(f"Total Due Amount: \n{extraction_result['total_due_amount']}\n")

print(f"Buyer Address: \n{extraction_result['buyer_address']}\n")

print(f"Ship To Address: \n{extraction_result['shipto_address']}\n")



Invoice Number: 
915378

Total Due Amount: 
749.37

Buyer Address: 
9 WEST 57TH STREET SUITE 4500 23 GRAND AVENUE NEW YORK , NY 10019 SHELTER ISLAND HGTS , NY 11965

Ship To Address: 
9 WEST 57TH STREET SUITE 4500 23 GRAND AVENUE NEW YORK , NY 10019 SHELTER ISLAND HGTS , NY 11965



### 2 Reviewer

####2.1  ReviewerLLM: Qwen 2.5 8B (Text only llm)

In [6]:
from utils.reviewer_utils import create_review_prompt_messages_text, parse_review


In [33]:
messages = create_review_prompt_messages_text(extraction_result, extracted_text)
response = llm_call(messages, model_identifier="qwen_25")
review_result = parse_review(response, extraction_result)
print("Review Result:")
print("-"*20)
print(review_result)
print("-"*20)


Using model: Qwen2.5-7B-Instruct


Parsing review response: {"buyer_address": {"status": "PASS", "feedback": ""}, "buyer_name": {"status": "PASS", "feedback": "...
Review Result:
--------------------
{'buyer_address': {'status': 'PASS', 'feedback': ''}, 'buyer_name': {'status': 'PASS', 'feedback': ''}, 'buyer_vat_number': {'status': 'PASS', 'feedback': ''}, 'currency': {'status': 'PASS', 'feedback': ''}, 'invoice_amount': {'status': 'PASS', 'feedback': ''}, 'invoice_date': {'status': 'PASS', 'feedback': ''}, 'invoice_number': {'status': 'PASS', 'feedback': ''}, 'payment_due_date': {'status': 'PASS', 'feedback': ''}, 'po_number': {'status': 'PASS', 'feedback': ''}, 'seller_address': {'status': 'PASS', 'feedback': ''}, 'seller_email': {'status': 'PASS', 'feedback': ''}, 'seller_fax_number': {'status': 'PASS', 'feedback': ''}, 'seller_name': {'status': 'PASS', 'feedback': ''}, 'seller_phone': {'status': 'PASS', 'feedback': ''}, 'seller_vat_number': {'status': 'PASS', 'feedback': ''}, 'seller_website': {'status': 'PASS', 'f

In [34]:
# Count passed and failed reviews
passed_count = sum(1 for v in review_result.values() if v['status'] == 'PASS')
failed_count = sum(1 for v in review_result.values() if v['status'] == 'FAIL')

# Print the counts
print(f"Passed: {passed_count}")
print(f"Failed: {failed_count}")
print("\nFailed items:")
print("-" * 20)

# Print failed items
for k, v in review_result.items():
    if v['status'] == 'FAIL':
        print(f"{k}: {v['feedback']}")

Passed: 22
Failed: 0

Failed items:
--------------------


In [ ]:
#Even Closed Source LLMs are Over confident.

####2.2 ReviewerLLM: Qwen 2.5 VL 7B (Multi Modal LLM)

In [7]:
from utils.reviewer_utils import create_review_prompt_messages, parse_review


In [35]:
messages = create_review_prompt_messages(extraction_result, extracted_text, img_bytes)
response = llm_call(messages, model_identifier="qwen_25_vl")
review_result = parse_review(response, extraction_result)
print("Review Result:")
print("-"*20)
print(review_result)
print("-"*20)

Using model: Qwen2.5-VL-7B-Instruct


Parsing review response: {
  "buyer_address": {"status": "FAIL", "feedback": "The address in the image is different from the ...
Review Result:
--------------------
{'buyer_address': {'status': 'FAIL', 'feedback': 'The address in the image is different from the extracted data.'}, 'buyer_name': {'status': 'PASS', 'feedback': ''}, 'buyer_vat_number': {'status': 'PASS', 'feedback': ''}, 'currency': {'status': 'PASS', 'feedback': ''}, 'invoice_amount': {'status': 'PASS', 'feedback': ''}, 'invoice_date': {'status': 'PASS', 'feedback': ''}, 'invoice_number': {'status': 'PASS', 'feedback': ''}, 'payment_due_date': {'status': 'PASS', 'feedback': ''}, 'po_number': {'status': 'PASS', 'feedback': ''}, 'seller_address': {'status': 'FAIL', 'feedback': 'The address in the image is different from the extracted data.'}, 'seller_email': {'status': 'PASS', 'feedback': ''}, 'seller_fax_number': {'status': 'PASS', 'feedback': ''}, 'seller_name': {'status': 'PASS', 'feedback': ''}, 'seller_phone': {'status

In [36]:
# Count passed and failed reviews
passed_count = sum(1 for v in review_result.values() if v['status'] == 'PASS')
failed_count = sum(1 for v in review_result.values() if v['status'] == 'FAIL')

# Print the counts
print(f"Passed: {passed_count}")
print(f"Failed: {failed_count}")
print("\nFailed items:")
print("-" * 20)

# Print failed items
for k, v in review_result.items():
    if v['status'] == 'FAIL':
        print(f"{k}: {v['feedback']}")

Passed: 19
Failed: 3

Failed items:
--------------------
buyer_address: The address in the image is different from the extracted data.
seller_address: The address in the image is different from the extracted data.
shipto_address: The address in the image is different from the extracted data.


In [ ]:
#2.3 ReviewerLLM: Qwen 3 (Reasoning LLM text only)

#### 2.3 ReviewerLLM: Qwen 3 (Reasoning LLM text only)


In [37]:
messages = create_review_prompt_messages_text(extraction_result, extracted_text)
response, reasoning = llm_call(messages, model_identifier="qwen_3")
review_result = parse_review(response, extraction_result)
print("Review Result:")
print("-"*20)
print(review_result)
print("-"*20)


Using model: Qwen/Qwen3-8B


Parsing review response: 

{"buyer_address": {"status": "PASS", "feedback": ""}, "buyer_name": {"status": "PASS", "feedback":...
Review Result:
--------------------
{'buyer_address': {'status': 'PASS', 'feedback': ''}, 'buyer_name': {'status': 'PASS', 'feedback': ''}, 'buyer_vat_number': {'status': 'PASS', 'feedback': ''}, 'currency': {'status': 'PASS', 'feedback': ''}, 'invoice_amount': {'status': 'PASS', 'feedback': ''}, 'invoice_date': {'status': 'PASS', 'feedback': ''}, 'invoice_number': {'status': 'PASS', 'feedback': ''}, 'payment_due_date': {'status': 'PASS', 'feedback': ''}, 'po_number': {'status': 'PASS', 'feedback': ''}, 'seller_address': {'status': 'PASS', 'feedback': ''}, 'seller_email': {'status': 'PASS', 'feedback': ''}, 'seller_fax_number': {'status': 'PASS', 'feedback': ''}, 'seller_name': {'status': 'PASS', 'feedback': ''}, 'seller_phone': {'status': 'PASS', 'feedback': ''}, 'seller_vat_number': {'status': 'PASS', 'feedback': ''}, 'seller_website': {'status': 'PASS', 'f

In [38]:
# Count passed and failed reviews
passed_count = sum(1 for v in review_result.values() if v['status'] == 'PASS')
failed_count = sum(1 for v in review_result.values() if v['status'] == 'FAIL')

# Print the counts
print(f"Passed: {passed_count}")
print(f"Failed: {failed_count}")
print("\nFailed items:")
print("-" * 20)

# Print failed items
for k, v in review_result.items():
    if v['status'] == 'FAIL':
        print(f"{k}: {v['feedback']}")

Passed: 22
Failed: 0

Failed items:
--------------------


In [17]:
print(reasoning)


Okay, let's start by looking at the extracted data and comparing each field against the OCR text provided. 

First, the buyer_address. The extracted value is a long string with multiple addresses. Looking at the OCR text, under "SOLD THE CHEQUIT" there's "9 WEST 57TH STREET SUITE 4500 23 GRAND AVENUE NEW YORK , NY 10019 SHELTER ISLAND HGTS , NY 11965". That seems to match the buyer_address exactly. So that's a PASS.

Next, buyer_name is "THE CHEQUIT". In the OCR text, right after "SOLD" it says "THE CHEQUIT", so that's correct. PASS.

Buyer_vat_number is empty. The document doesn't mention any VAT number for the buyer, so PASS.

Currency is USD. The invoice is in USD, as indicated by the amounts and the context. PASS.

Invoice_amount is 749.37. The total at the end of the OCR text says "Total 749.37", so that's correct. PASS.

Invoice_date is 08/07/24. The OCR text has "Invoice Date 08/07/24", so the format is correct. PASS.

Invoice_number is 915378. The OCR text has "Invoice No. 915

In [ ]:
print(f"""Looking at the OCR text,
there's a section under "SOLD THE CHEQUIT" followed by the addresses: "9 WEST 57TH STREET SUITE 4500 23 GRAND AVENUE NEW YORK , NY 10019 SHELTER ISLAND HGTS , NY 11965".
That seems to match the buyer_address exactly.
But wait, the OCR text also has "SHIP THE CHEQUIT" next to the address.
Maybe that's part of the shipto address?
But the extracted data has the same address for both buyer_address and shipto_address.
I need to check if the buyer_address is correctly captured....""")

Looking at the OCR text, 
there's a section under "SOLD THE CHEQUIT" followed by the addresses: "9 WEST 57TH STREET SUITE 4500 23 GRAND AVENUE NEW YORK , NY 10019 SHELTER ISLAND HGTS , NY 11965". 
That seems to match the buyer_address exactly. 
But wait, the OCR text also has "SHIP THE CHEQUIT" next to the address. 
Maybe that's part of the shipto address? 
But the extracted data has the same address for both buyer_address and shipto_address. 
I need to check if the buyer_address is correctly captured....


## Iteration 2
####1. Extractor LLM: Qwen2.5 VL 7B
#####Multimodal LLM (Vision Encoder + Decoder)

In [8]:
from utils.extractor_utils import create_extraction_prompt_messages

In [40]:
messages = create_extraction_prompt_messages(extracted_fields, extracted_text, img_bytes)
response = llm_call(messages, model_identifier="qwen_25_vl")
extraction_result = parse_extraction(response, extracted_fields)
print("Extraction Result:")
print("-"*20)
print(extraction_result)
print("-"*20)

Using model: Qwen2.5-VL-7B-Instruct


Extraction Result:
--------------------
{'buyer_address': '9 WEST 57TH STREET SUITE 4500 NEW YORK, NY 10019', 'buyer_name': 'THE CHEQUIT', 'buyer_vat_number': '', 'currency': 'USD', 'invoice_amount': '742.37', 'invoice_date': '08/06/24', 'invoice_number': '915378', 'payment_due_date': '08/07/24', 'po_number': '', 'seller_address': '210 Main Street Farmingdale, NY 11735', 'seller_email': '', 'seller_fax_number': '', 'seller_name': 'Main Street Wholesale Meats', 'seller_phone': '(516) 249-8200', 'seller_vat_number': '', 'seller_website': '', 'shipping_date': '', 'shipto_address': '23 GRAND AVENUE SHELTER ISLAND HGTS, NY 11965', 'shipto_name': 'THE CHEQUIT', 'subtotal': '742.37', 'total_due_amount': '749.37', 'total_tax': '7.00'}
--------------------


In [41]:
print(f"Invoice Number: \n{extraction_result['invoice_number']}\n")

print(f"Total Due Amount: \n{extraction_result['total_due_amount']}\n")

print(f"Buyer Address: \n{extraction_result['buyer_address']}\n")

print(f"Ship To Address: \n{extraction_result['shipto_address']}\n")

Invoice Number: 
915378

Total Due Amount: 
749.37

Buyer Address: 
9 WEST 57TH STREET SUITE 4500 NEW YORK, NY 10019

Ship To Address: 
23 GRAND AVENUE SHELTER ISLAND HGTS, NY 11965



###2. Reviewer

####2.1 Reviewer LLM: Qwen 2.5

In [ ]:
messages = create_review_prompt_messages_text(extraction_result, extracted_text)
response = llm_call(messages, model_identifier="qwen_25")
review_result = parse_review(response, extraction_result)
print("Review Result:")
print("-"*20)
print(review_result)
print("-"*20)


Using model: Qwen2.5-7B-Instruct


Parsing review response: {"buyer_address": {"status": "PASS", "feedback": ""}, "buyer_name": {"status": "PASS", "feedback": "...
Review Result:
--------------------
{'buyer_address': {'status': 'PASS', 'feedback': ''}, 'buyer_name': {'status': 'PASS', 'feedback': ''}, 'buyer_vat_number': {'status': 'PASS', 'feedback': ''}, 'currency': {'status': 'PASS', 'feedback': ''}, 'invoice_amount': {'status': 'PASS', 'feedback': ''}, 'invoice_date': {'status': 'PASS', 'feedback': ''}, 'invoice_number': {'status': 'PASS', 'feedback': ''}, 'payment_due_date': {'status': 'PASS', 'feedback': ''}, 'po_number': {'status': 'PASS', 'feedback': ''}, 'seller_address': {'status': 'PASS', 'feedback': ''}, 'seller_email': {'status': 'PASS', 'feedback': ''}, 'seller_fax_number': {'status': 'PASS', 'feedback': ''}, 'seller_name': {'status': 'PASS', 'feedback': ''}, 'seller_phone': {'status': 'PASS', 'feedback': ''}, 'seller_vat_number': {'status': 'PASS', 'feedback': ''}, 'seller_website': {'status': 'PASS', 'f

In [ ]:
# Count passed and failed reviews
passed_count = sum(1 for v in review_result.values() if v['status'] == 'PASS')
failed_count = sum(1 for v in review_result.values() if v['status'] == 'FAIL')

# Print the counts
print(f"Passed: {passed_count}")
print(f"Failed: {failed_count}")
print("\nFailed items:")
print("-" * 20)

# Print failed items
for k, v in review_result.items():
    if v['status'] == 'FAIL':
        print(f"{k}: {v['feedback']}")

Passed: 22
Failed: 0

Failed items:
--------------------


####2.2 Reviewer LLM: Qwen 2.5 VL

In [42]:
messages = create_review_prompt_messages(extraction_result, extracted_text, img_bytes)
response = llm_call(messages, model_identifier="qwen_25_vl")
review_result = parse_review(response, extraction_result)
print("Review Result:")
print("-"*20)
print(review_result)
print("-"*20)


Using model: Qwen2.5-VL-7B-Instruct


Parsing review response: {
  "buyer_address": {"status": "PASS", "feedback": ""},
  "buyer_name": {"status": "PASS", "feedbac...
Review Result:
--------------------
{'buyer_address': {'status': 'PASS', 'feedback': ''}, 'buyer_name': {'status': 'PASS', 'feedback': ''}, 'buyer_vat_number': {'status': 'PASS', 'feedback': ''}, 'currency': {'status': 'PASS', 'feedback': ''}, 'invoice_amount': {'status': 'PASS', 'feedback': ''}, 'invoice_date': {'status': 'PASS', 'feedback': ''}, 'invoice_number': {'status': 'PASS', 'feedback': ''}, 'payment_due_date': {'status': 'PASS', 'feedback': ''}, 'po_number': {'status': 'PASS', 'feedback': ''}, 'seller_address': {'status': 'PASS', 'feedback': ''}, 'seller_email': {'status': 'PASS', 'feedback': ''}, 'seller_fax_number': {'status': 'PASS', 'feedback': ''}, 'seller_name': {'status': 'PASS', 'feedback': ''}, 'seller_phone': {'status': 'PASS', 'feedback': ''}, 'seller_vat_number': {'status': 'PASS', 'feedback': ''}, 'seller_website': {'status': 'PASS', 'f

In [43]:
# Count passed and failed reviews
passed_count = sum(1 for v in review_result.values() if v['status'] == 'PASS')
failed_count = sum(1 for v in review_result.values() if v['status'] == 'FAIL')

# Print the counts
print(f"Passed: {passed_count}")
print(f"Failed: {failed_count}")
print("\nFailed items:")
print("-" * 20)

# Print failed items
for k, v in review_result.items():
    if v['status'] == 'FAIL':
        print(f"{k}: {v['feedback']}")

Passed: 22
Failed: 0

Failed items:
--------------------


####2.3 Reviewer LLM: Qwen 3 (Reasoning LLM text only)

In [44]:
messages = create_review_prompt_messages_text(extraction_result, extracted_text)
response, reasoning = llm_call(messages, model_identifier="qwen_3")
review_result = parse_review(response, extraction_result)
print("Review Result:")
print("-"*20)
print(review_result)
print("-"*20)


Using model: Qwen/Qwen3-8B


Parsing review response: 

{"buyer_address": {"status": "PASS", "feedback": ""}, "buyer_name": {"status": "PASS", "feedback":...
Review Result:
--------------------
{'buyer_address': {'status': 'PASS', 'feedback': ''}, 'buyer_name': {'status': 'PASS', 'feedback': ''}, 'buyer_vat_number': {'status': 'PASS', 'feedback': ''}, 'currency': {'status': 'PASS', 'feedback': ''}, 'invoice_amount': {'status': 'PASS', 'feedback': ''}, 'invoice_date': {'status': 'FAIL', 'feedback': 'Incorrect date format (extracted 08/06/24 vs document 08/07/24)'}, 'invoice_number': {'status': 'PASS', 'feedback': ''}, 'payment_due_date': {'status': 'PASS', 'feedback': ''}, 'po_number': {'status': 'FAIL', 'feedback': 'Value not found in document (PO Number 1 present)'}, 'seller_address': {'status': 'PASS', 'feedback': ''}, 'seller_email': {'status': 'PASS', 'feedback': ''}, 'seller_fax_number': {'status': 'PASS', 'feedback': ''}, 'seller_name': {'status': 'PASS', 'feedback': ''}, 'seller_phone': {'status': 'PASS', '

In [45]:
# Count passed and failed reviews
passed_count = sum(1 for v in review_result.values() if v['status'] == 'PASS')
failed_count = sum(1 for v in review_result.values() if v['status'] == 'FAIL')

# Print the counts
print(f"Passed: {passed_count}")
print(f"Failed: {failed_count}")
print("\nFailed items:")
print("-" * 20)

# Print failed items
for k, v in review_result.items():
    if v['status'] == 'FAIL':
        print(f"{k}: {v['feedback']}")

Passed: 20
Failed: 2

Failed items:
--------------------
invoice_date: Incorrect date format (extracted 08/06/24 vs document 08/07/24)
po_number: Value not found in document (PO Number 1 present)


In [ ]:
extraction_result['invoice_date']

'08/06/24'

In [ ]:
print(reasoning)


Okay, let's start by carefully reviewing the extracted data against the OCR text provided. The goal is to check each field in the JSON to see if it matches the information in the OCR text. 

First, the buyer_address is "9 WEST 57TH STREET SUITE 4500 NEW YORK, NY 10019". Looking through the OCR text, I see "9 WEST 57TH STREET SUITE 4500 NEW YORK , NY 10019" under the "SOLD THE CHEQUIT" section. The address is present, so that's a PASS.

Next, buyer_name is "THE CHEQUIT". The OCR text mentions "SOLD THE CHEQUIT" which matches, so that's a PASS. 

Buyer_vat_number is empty. The OCR text doesn't mention any VAT number for the buyer, so that's a PASS.

Currency is USD. The OCR text doesn't explicitly state the currency, but since it's an invoice in the US and the amounts are in dollars, USD is likely correct. However, the OCR text doesn't mention USD, so maybe this is a FAIL? Wait, the total amounts are in USD, but the currency field might be inferred. However, the instruction says to chec

In [ ]:
print("""Currency is USD. The OCR text doesn't explicitly state the currency,
but since it's an invoice in the US and the amounts are in dollars, USD is likely correct.
However, the OCR text doesn't mention USD, so maybe this is a FAIL? Wait, the total amounts are in USD, but the currency field might be inferred.
However, the instruction says to check against the document. Since the OCR text doesn't have the currency specified, maybe this is a FAIL.
Wait, the user might have assumed USD based on the context, but the extracted data has USD. The OCR text doesn't have the currency explicitly, so maybe this is a FAIL. Hmm, need to check again. The OCR text has amounts like 742.37, which are in USD, but the currency isn't stated. So the extracted value is USD, but the document doesn't have it. So this would be a FAIL? Or is it acceptable to assume USD? The instruction says to check against the document. If the document doesn't have the currency, then the extracted value might be incorrect. But since the user provided the OCR text and the extracted data includes USD, which is the standard for the US, maybe it's a PASS. However, the OCR text doesn't explicitly state the currency. This is a bit ambiguous. I'll have to check other fields for similar issues.

Invoice_date is 08/06/24.
The OCR text mentions "Invoice Date 08/07/24" but the invoice date might be the date of the invoice,
which could be the same as the payment due date. Wait, the OCR text has "Invoice No. 915378 08/07/24" which is the invoice date.
The extracted data has 08/06/24 as the invoice date. Wait, the extracted data's invoice_date is 08/06/24, but the OCR text shows the invoice date as 08/07/24.
So this is a discrepancy. Therefore, this is a FAIL. The invoice_date in the extracted data is incorrect.""")

Currency is USD. The OCR text doesn't explicitly state the currency, 
but since it's an invoice in the US and the amounts are in dollars, USD is likely correct. 
However, the OCR text doesn't mention USD, so maybe this is a FAIL? Wait, the total amounts are in USD, but the currency field might be inferred. 
However, the instruction says to check against the document. Since the OCR text doesn't have the currency specified, maybe this is a FAIL. 
Wait, the user might have assumed USD based on the context, but the extracted data has USD. The OCR text doesn't have the currency explicitly, so maybe this is a FAIL. Hmm, need to check again. The OCR text has amounts like 742.37, which are in USD, but the currency isn't stated. So the extracted value is USD, but the document doesn't have it. So this would be a FAIL? Or is it acceptable to assume USD? The instruction says to check against the document. If the document doesn't have the currency, then the extracted value might be incorrect. But s

### Future: Visual Reasoning

## AGENT in Action

In [3]:
from qwen_agent.utils.output_beautify import typewriter_print
from document_ai_agent import Assistant, LLM_CONFIG, SYSTEM_INSTRUCTION
import base64
import os


In [4]:
bot = Assistant(
    llm=LLM_CONFIG,
    system_message=SYSTEM_INSTRUCTION,
    function_list=['extractor_ai', 'reviewer_ai']
)

In [ ]:
#Inputs: Image, OCR text, fields to extract
messages = []
ocr_text_input = extracted_text
fields_str_input = str(extracted_fields)

if not os.path.exists(image_path):
    print(f"Error: Image file not found at {image_path}")
    raise Exception(f"Image file not found at {image_path}")

try:
    with open(image_path, "rb") as f:
        image_bytes = f.read()
    image_bytes_b64 = base64.b64encode(image_bytes).decode('utf-8')
    
except Exception as e:
    print(f"Error reading or encoding image file: {e}")
    raise e

try:
    temp_dir = "_temp_doc_ai_files"
    temp_image_filename = "_temp_doc_ai_files/_image_data.b64"
    with open(temp_image_filename, "w") as f_img_temp:
        f_img_temp.write(image_bytes_b64)
    temp_ocr_filename = "_temp_doc_ai_files/_ocr_text.txt"
    with open(temp_ocr_filename, "w") as f_ocr_temp:
        f_ocr_temp.write(ocr_text_input)
except Exception as e_write_temp:
    print(f"Error writing to temporary files: {e_write_temp}")
    raise e_write_temp

In [16]:
query = (
    f"Please process the document. "
    f"The image data is in the file named '{os.path.basename(temp_image_filename)}' and "
    f"the OCR text is in the file named '{os.path.basename(temp_ocr_filename)}'. "
    f"These files are located in the '{temp_dir}' directory relative to my current location. " # Giving relative path context
    f"So, the path for the image file is '{temp_image_filename}' and for the OCR text is '{temp_ocr_filename}'. "
    f"Fields to Extract (JSON array string): '{fields_str_input}'"
)


messages.append({'role': 'user', 'content': query})
run_messages = messages 

print("\nBot response:")
response_plain_text = ''
full_bot_response = []
for response_chunk in bot.run(messages=messages):
    # Streaming output
    response_plain_text = typewriter_print(response_chunk, response_plain_text)
    full_bot_response.extend(response_chunk)

messages.extend(full_bot_response)
print("\n--- End of Bot Response ---")



Bot response:
[THINK]

Okay, let's see

. The user wants me to process a document. They provided the image data file as '_image_data.b64' and the OCR text as '_ocr_text.txt', both in the '_temp_doc_ai_files' directory. The fields they need extracted are a list of various addresses, names, VAT numbers, etc.

First, I need to determine if this is the extraction stage or review stage. Since there's no previous extracted JSON mentioned, it's the first turn, so extraction stage. 

I should call the extractor_ai tool. The parameters required are image_file_path, ocr_file_path, fields_to_extract, and llm_name. The user specified the paths as '_temp_doc_ai_files/_image_data.b64' and '_temp_doc_ai_files/_ocr_text.txt'. The fields are given as a JSON array string. The preferred LLM is 'qwen_25_vl' for vision capabilities.

Wait, the fields_to_extract in the user's message is a JSON array string. Let me check the format. The user wrote: Fields to Extract (JSON array string): '['buyer_address', ...]'. But the actual string might have es

In [18]:
"""
Bot response:
[THINK]

Okay, let's see
. The user wants me to process a document. They provided the image data file as '_image_data.b64' and the OCR text as '_ocr_text.txt', both in the '_temp_doc_ai_files' directory. The fields they need extracted are a list of various addresses, names, VAT numbers, etc.

First, I need to determine if this is the extraction stage or review stage. Since there's no previous extracted JSON mentioned, it's the first turn, so extraction stage. 

I should call the extractor_ai tool. The parameters required are image_file_path, ocr_file_path, fields_to_extract, and llm_name. The user specified the paths as '_temp_doc_ai_files/_image_data.b64' and '_temp_doc_ai_files/_ocr_text.txt'. The fields are given as a JSON array string. The preferred LLM is 'qwen_25_vl' for vision capabilities.

Wait, the fields_to_extract in the user's message is a JSON array string. Let me check the format. The user wrote: Fields to Extract (JSON array string): '['buyer_address', ...]'. But the actual string might have escaped quotes. I need to make sure the fields_to_extract parameter is correctly formatted as a JSON string array. The user's input seems to have the fields in a string that's already a JSON array, so I can pass it as-is. 

So, the function call would be to extractor_ai with those paths, the fields, and llm_name 'qwen_25_vl'. That should extract the necessary fields. Then, after extraction, the next step would be review, but the user hasn't provided any extracted data yet. So the first step is definitely the extractor_ai.




[TOOL_CALL] extractor_ai
{"image_file_path": "_temp_doc_ai_files/_image_data.b64", "ocr_file_path": "_temp_doc_ai_files/_ocr_text.txt", "fields_to_extract": "[\"buyer_address\", \"buyer_name\", \"buyer_vat_number\", \"currency\", \"invoice_amount\", \"invoice_date\", \"invoice_number\", \"payment_due_date\", \"po_number\", \"seller_address\", \"seller_email\", \"seller_fax_number\", \"seller_name\", \"seller_phone\", \"seller_vat_number\", \"seller_website\", \"shipping_date\", \"shipto_address\", \"shipto_name\", \"subtotal\", \"total_due_amount\", \"total_tax\"]", "llm_name": "qwen_25_vl"}--- ExtractorAI called with LLM: qwen_25_vl ---
Reading image data from: _temp_doc_ai_files/_image_data.b64
Reading OCR text from: _temp_doc_ai_files/_ocr_text.txt
ExtractorAI - OCR Text (first 100 chars): WHOLESALE
Main Street Wholesale Meats INVOICE
Farmingdale Meat Market
FARMINGDALE 210 Main Street
MA
ExtractorAI - Fields to extract: ["buyer_address", "buyer_name", "buyer_vat_number", "currency", "invoice_amount", "invoice_date", "invoice_number", "payment_due_date", "po_number", "seller_address", "seller_email", "seller_fax_number", "seller_name", "seller_phone", "seller_vat_number", "seller_website", "shipping_date", "shipto_address", "shipto_name", "subtotal", "total_due_amount", "total_tax"]
ExtractorAI - Parsed fields to extract: ['buyer_address', 'buyer_name', 'buyer_vat_number', 'currency', 'invoice_amount', 'invoice_date', 'invoice_number', 'payment_due_date', 'po_number', 'seller_address', 'seller_email', 'seller_fax_number', 'seller_name', 'seller_phone', 'seller_vat_number', 'seller_website', 'shipping_date', 'shipto_address', 'shipto_name', 'subtotal', 'total_due_amount', 'total_tax']
Using model: internal_vlm_v1
ExtractorAI - LLM call successful, Raw LLM Response (first 100 chars): ```json
{
  "buyer_address": "9 WEST 57TH STREET\nSUITE 4500",
  "buyer_name": "THE CHEQUIT",
  "buy...
ExtractorAI - Parsed Extraction Result: {'buyer_address': '9 WEST 57TH STREET\nSUITE 4500', 'buyer_name': 'THE CHEQUIT', 'buyer_vat_number': '', 'currency': 'USD', 'invoice_amount': '749.37', 'invoice_date': '08/07/24', 'invoice_number': '915378', 'payment_due_date': '', 'po_number': '1', 'seller_address': '210 Main Street\nFarmingdale , NY 11735', 'seller_email': '', 'seller_fax_number': '', 'seller_name': 'Main Street Wholesale Meats', 'seller_phone': '(516) 249-8200', 'seller_vat_number': '', 'seller_website': '', 'shipping_date': '', 'shipto_address': '23 GRAND AVENUE\nSHELTER ISLAND HGTS , NY 11965', 'shipto_name': 'THE CHEQUIT', 'subtotal': '742.37', 'total_due_amount': '749.37', 'total_tax': '7.00'}

[TOOL_RESPONSE] extractor_ai
{buyer_address: "9 WEST 57TH STREET\nSUITE 4500", buyer_name: "THE CHEQUIT", buyer_vat_number: "", currency: "USD", invoice_amount: "749.37", invoice_date: "08/07/24", invoice_number: "915378", payment_due_date: "", po_number: "1", seller_address: "210 Main Street\nFarmingdale , NY 11735", seller_email: "", seller_fax_number: "", seller_name: "Main Street Wholesale Meats", seller_phone: "(516) 249-8200", seller_vat_number: "", seller_website: "", shipping_date: "", shipto_address: "23 GRAND AVENUE\nSHELTER ISLAND HGTS , NY 11965", shipto_name: "THE CHEQUIT", subtotal: "742.37", total_due_amount: "749.37", total_tax: "7.00"}
[THINK]

Okay, let me start by understanding what the user needs. They provided an image and OCR text file paths, along with a list of fields to extract from a document. The first step was to call the extractor_ai tool with the given parameters. The response from the tool gave me the extracted data. Now, since there's extracted JSON output, this is the review stage.

Next, I need to use the reviewer_ai tool to check the extracted data against the original documents. The parameters required are the image file path, OCR text path, the extracted JSON string, and the LLM name. The user mentioned using "qwen_25_vl" for extraction, but the reviewer_ai options are "qwen_25_vl" or "qwen_3". I should use "qwen_25_vl" again for consistency, as it's better for vision tasks.

I need to make sure all the fields listed are reviewed. The extracted data has some empty fields like buyer_vat_number and seller_vat_number. The reviewer_ai will check if these are correctly identified as empty or if they should have values. Also, dates and numbers should be verified for accuracy. For example, the invoice_amount and total_due_amount are both "749.37", which seems correct, but the reviewer should confirm. Similarly, the subtotal is "742.37" and total_tax is "7.00", which adds up correctly. 

I should also check if the addresses and names are correctly captured. The shipto_address and buyer_address might have formatting issues, like the newline characters. The reviewer_ai will assess if these are properly handled. Additionally, the payment_due_date and shipping_date are empty; the reviewer needs to determine if that's accurate based on the document.

Once the reviewer_ai provides the status for each field, I'll compile the final output. If all fields are confirmed correct, the status is PASS. If any discrepancies are found, they'll be marked as FAIL. The final JSON should list each field with its value and status. I need to ensure the structure is correct and only the required fields are included.




[TOOL_CALL] reviewer_ai
{"image_file_path": "_temp_doc_ai_files/_image_data.b64", "ocr_file_path": "_temp_doc_ai_files/_ocr_text.txt", "extracted_json_str": "{\"buyer_address\": \"9 WEST 57TH STREET\\nSUITE 4500\", \"buyer_name\": \"THE CHEQUIT\", \"buyer_vat_number\": \"\", \"currency\": \"USD\", \"invoice_amount\": \"749.37\", \"invoice_date\": \"08/07/24\", \"invoice_number\": \"915378\", \"payment_due_date\": \"\", \"po_number\": \"1\", \"seller_address\": \"210 Main Street\\nFarmingdale , NY 11735\", \"seller_email\": \"\", \"seller_fax_number\": \"\", \"seller_name\": \"Main Street Wholesale Meats\", \"seller_phone\": \"(516) 249-8200\", \"seller_vat_number\": \"\", \"seller_website\": \"\", \"shipping_date\": \"\", \"shipto_address\": \"23 GRAND AVENUE\\nSHELTER ISLAND HGTS , NY 11965\", \"shipto_name\": \"THE CHEQUIT\", \"subtotal\": \"742.37\", \"total_due_amount\": \"749.37\", \"total_tax\": \"7.00\"}", "llm_name": "qwen_25_vl"}--- ReviewerAI called with LLM: qwen_25_vl ---
Reading image data from: _temp_doc_ai_files/_image_data.b64
Reading OCR text from: _temp_doc_ai_files/_ocr_text.txt
Reading extracted JSON from: {"buyer_address": "9 WEST 57TH STREET\nSUITE 4500", "buyer_name": "THE CHEQUIT", "buyer_vat_number": "", "currency": "USD", "invoice_amount": "749.37", "invoice_date": "08/07/24", "invoice_number": "915378", "payment_due_date": "", "po_number": "1", "seller_address": "210 Main Street\nFarmingdale , NY 11735", "seller_email": "", "seller_fax_number": "", "seller_name": "Main Street Wholesale Meats", "seller_phone": "(516) 249-8200", "seller_vat_number": "", "seller_website": "", "shipping_date": "", "shipto_address": "23 GRAND AVENUE\nSHELTER ISLAND HGTS , NY 11965", "shipto_name": "THE CHEQUIT", "subtotal": "742.37", "total_due_amount": "749.37", "total_tax": "7.00"}
ReviewerAI - Parsed extracted JSON for review: {'buyer_address': '9 WEST 57TH STREET\nSUITE 4500', 'buyer_name': 'THE CHEQUIT', 'buyer_vat_number': '', 'currency': 'USD', 'invoice_amount': '749.37', 'invoice_date': '08/07/24', 'invoice_number': '915378', 'payment_due_date': '', 'po_number': '1', 'seller_address': '210 Main Street\nFarmingdale , NY 11735', 'seller_email': '', 'seller_fax_number': '', 'seller_name': 'Main Street Wholesale Meats', 'seller_phone': '(516) 249-8200', 'seller_vat_number': '', 'seller_website': '', 'shipping_date': '', 'shipto_address': '23 GRAND AVENUE\nSHELTER ISLAND HGTS , NY 11965', 'shipto_name': 'THE CHEQUIT', 'subtotal': '742.37', 'total_due_amount': '749.37', 'total_tax': '7.00'}
Using model: internal_vlm_v1
Parsing review response: {
  "buyer_address": {"status": "PASS", "feedback": ""},
  "buyer_name": {"status": "PASS", "feedbac...
ReviewerAI - LLM call successful, Raw LLM Response (first 100 chars): {
  "buyer_address": {"status": "PASS", "feedback": ""},
  "buyer_name": {"status": "PASS", "feedbac...
ReviewerAI - Parsed Review Result: {'buyer_address': {'status': 'PASS', 'feedback': ''}, 'buyer_name': {'status': 'PASS', 'feedback': ''}, 'buyer_vat_number': {'status': 'PASS', 'feedback': ''}, 'currency': {'status': 'PASS', 'feedback': ''}, 'invoice_amount': {'status': 'PASS', 'feedback': ''}, 'invoice_date': {'status': 'PASS', 'feedback': ''}, 'invoice_number': {'status': 'PASS', 'feedback': ''}, 'payment_due_date': {'status': 'PASS', 'feedback': ''}, 'po_number': {'status': 'PASS', 'feedback': ''}, 'seller_address': {'status': 'PASS', 'feedback': ''}, 'seller_email': {'status': 'PASS', 'feedback': ''}, 'seller_fax_number': {'status': 'PASS', 'feedback': ''}, 'seller_name': {'status': 'PASS', 'feedback': ''}, 'seller_phone': {'status': 'PASS', 'feedback': ''}, 'seller_vat_number': {'status': 'PASS', 'feedback': ''}, 'seller_website': {'status': 'PASS', 'feedback': ''}, 'shipping_date': {'status': 'PASS', 'feedback': ''}, 'shipto_address': {'status': 'PASS', 'feedback': ''}, 'shipto_name': {'status': 'PASS', 'feedback': ''}, 'subtotal': {'status': 'PASS', 'feedback': ''}, 'total_due_amount': {'status': 'PASS', 'feedback': ''}, 'total_tax': {'status': 'PASS', 'feedback': ''}}

[TOOL_RESPONSE] reviewer_ai
{buyer_address: {status: "PASS", feedback: ""}, buyer_name: {status: "PASS", feedback: ""}, buyer_vat_number: {status: "PASS", feedback: ""}, currency: {status: "PASS", feedback: ""}, invoice_amount: {status: "PASS", feedback: ""}, invoice_date: {status: "PASS", feedback: ""}, invoice_number: {status: "PASS", feedback: ""}, payment_due_date: {status: "PASS", feedback: ""}, po_number: {status: "PASS", feedback: ""}, seller_address: {status: "PASS", feedback: ""}, seller_email: {status: "PASS", feedback: ""}, seller_fax_number: {status: "PASS", feedback: ""}, seller_name: {status: "PASS", feedback: ""}, seller_phone: {status: "PASS", feedback: ""}, seller_vat_number: {status: "PASS", feedback: ""}, seller_website: {status: "PASS", feedback: ""}, shipping_date: {status: "PASS", feedback: ""}, shipto_address: {status: "PASS", feedback: ""}, shipto_name: {status: "PASS", feedback: ""}, subtotal: {status: "PASS", feedback: ""}, total_due_amount: {status: "PASS", feedback: ""}, total_tax: {status: "PASS", feedback: ""}}
[THINK]

Okay, let's see. The user provided the image and OCR files, and the fields they want extracted. First, I called the extractor_ai with the right parameters. The response had all the fields filled, but some were empty like buyer_vat_number. Then I used the reviewer_ai to check those extracted fields against the documents. The reviewer returned all PASS with no feedback. That means the extracted data matches the documents correctly. Even though some fields were empty in the OCR, the reviewer didn't flag them, so they must be correct as per the documents. Now, I need to compile the final output JSON with the extracted data and the review statuses. All fields are PASS, so the final output should reflect that.



FINAL_OUTPUT_JSON: {"buyer_address": {"value": "9 WEST 57TH STREET\nSUITE 4500", "status": "PASS"}, "buyer_name": {"value": "THE CHEQUIT", "status": "PASS"}, "buyer_vat_number": {"value": "", "status": "PASS"}, "currency": {"value": "USD", "status": "PASS"}, "invoice_amount": {"value": "749.37", "status": "PASS"}, "invoice_date": {"value": "08/07/24", "status": "PASS"}, "invoice_number": {"value": "915378", "status": "PASS"}, "payment_due_date": {"value": "", "status": "PASS"}, "po_number": {"value": "1", "status": "PASS"}, "seller_address": {"value": "210 Main Street\nFarmingdale , NY 11735", "status": "PASS"}, "seller_email": {"value": "", "status": "PASS"}, "seller_fax_number": {"value": "", "status": "PASS"}, "seller_name": {"value": "Main Street Wholesale Meats", "status": "PASS"}, "seller_phone": {"value": "(516) 249-8200", "status": "PASS"}, "seller_vat_number": {"value": "", "status": "PASS"}, "seller_website": {"value": "", "status": "PASS"}, "shipping_date": {"value": "", "status": "PASS"}, "shipto_address": {"value": "23 GRAND AVENUE\nSHELTER ISLAND HGTS , NY 11965", "status": "PASS"}, "shipto_name": {"value": "THE CHEQUIT", "status": "PASS"}, "subtotal": {"value": "742.37", "status": "PASS"}, "total_due_amount": {"value": "749.37", "status": "PASS"}, "total_tax": {"value": "7.00", "status": "PASS"}}
--- End of Bot Response ---
"""

'\n. The user wants me to process a document. They provided the image data file as \'_image_data.b64\' and the OCR text as \'_ocr_text.txt\', both in the \'_temp_doc_ai_files\' directory. The fields they need extracted are a list of various addresses, names, VAT numbers, etc.\n\nFirst, I need to determine if this is the extraction stage or review stage. Since there\'s no previous extracted JSON mentioned, it\'s the first turn, so extraction stage. \n\nI should call the extractor_ai tool. The parameters required are image_file_path, ocr_file_path, fields_to_extract, and llm_name. The user specified the paths as \'_temp_doc_ai_files/_image_data.b64\' and \'_temp_doc_ai_files/_ocr_text.txt\'. The fields are given as a JSON array string. The preferred LLM is \'qwen_25_vl\' for vision capabilities.\n\nWait, the fields_to_extract in the user\'s message is a JSON array string. Let me check the format. The user wrote: Fields to Extract (JSON array string): \'[\'buyer_address\', ...]\'. But the